# Pixel Sorter
I found a simple pixel sorting algorithm written in 2014 by ProphetGoddess (PG): https://gist.github.com/prophetgoddess/667c5554e5d9d9a25ae6 which provides 3 sorting methods: absolute, random row sort, and pivot sort.

Each of these methods rely on PG's `quick_sort` algorithm which iteratively removes and appends pixels in a row depending on the pixel brightness. The use of Python lists in this algo results in a painfully slow process, so I chose this as the first place to implement an improvement. By relying on the vectorized properties and "in-place" methods of `numpy` I can expect a heafty speedup.

The libraries I need to use are the P(ython)I(mage)L(ibrary) and NumPy.

In [9]:
from PIL import Image
import numpy as np

First, I can read-in the image data as a 3xN array of RGB values for each of N pixels.

In [53]:
im = Image.open('./sonoshee_wp.png', 'r')
pix_val = np.array(im.getdata())
print(pix_val)
print("pixel data shape:",pix_val.shape)
print("image dimensions:",im.size)

[139 139 139 ...  13  15  15]
pixel data shape: (2073600,)
image dimensions: (1920, 1080)


Next I want to sort the pixel dimensions based on a function of each pixels' RGB values. To do this, I first need to create a copy of the array, perform the RGB function (effectively zipping the (N\*M,3) array into (N\*M,1) then reorganize the original pixel data based on the sort order of this collapsed array.

This method of sorting was inspired by [this stackoverflow post](https://stackoverflow.com/questions/1903462/how-can-i-zip-sort-parallel-numpy-arrays).

TODO: I'd like to pass an option to flip the pixel data and its collapsed copy so that I can choose to sort by column or row.

In [43]:
def quicker_sort(pixels):
    pixels = np.array(pixels)
    pix_cpy = pixels.copy()
    pix_cpy = pix_cpy.sum(axis=1)
    return pixels[pix_cpy.argsort()]

sorted_pix_val = quicker_sort(pix_val)
print("reshaped pixel data:", sorted_pix_val.shape)
print(sorted_pix_val)

(1920000,) [275 275 275 ... 275 275 275]
reshaped pixel data: (1920000, 3)
[[119   0   0]
 [119   0   0]
 [  0 119   0]
 ...
 [221 255 238]
 [221 255 238]
 [221 255 238]]


I want to make sure the pixel data actually reflects the true dimensionality/organization of the source image. I can reshape the pixel data accordingly, resulting in a 3D NxMx3 matrix.

In [44]:
colrow = sorted_pix_val.reshape(im.size[0],im.size[1],3)
print("reshaped pixel data:", colrow.shape)

reshaped pixel data: (1200, 1600, 3)


In [45]:
new = Image.fromarray(colrow.astype('uint8'), 'RGB')
new.save('test_sort.png')

This works great, but is really just a long ordered list of pixels (which makes sense as all I've done is  sort the pixels as a massive (N\*M,3) array and reshaped into the orignial image shape. Let's instead try to do this sorting for each row of the source image.

In [94]:
im = Image.open('./test_sort.png', 'r')
pix_val = np.array(im.getdata())
colrow = pix_val.reshape(im.size[0],im.size[1],3)

sort = quicker_sort(colrow[:,0])
print(sort.shape, sort)

(1600,) [275 275 275 ... 275 275 275]
(1600, 3) [[  0  46 229]
 [  0  46 229]
 [  0  46 229]
 ...
 [  0  46 229]
 [  0  46 229]
 [  0  46 229]]


In [95]:
new = Image.fromarray(colrow.astype('uint8'), 'RGB')
new.save('test_sort.png')

In [96]:
print(colrow[:,0].shape)

(1600, 3)


In [46]:
from PIL import Image
import numpy as np


In [4]:
im = Image.open('/Users/aleph/Downloads/IMG_4634.JPEG', 'r')
pix_val = np.array(im.getdata())

In [3]:
print(type(pix_val))
print(pix_val.shape)
print(im.size)

<class 'numpy.ndarray'>
(349700, 3)
(650, 538)


In [4]:
colrow = pix_val.reshape(im.size[0],im.size[1],3)

In [5]:
print(colrow.shape)

(650, 538, 3)


In [6]:
# This is a single column
print(colrow[0][0])
def calc_lum(a):
    # Assumes a is of shape (3,)
    return np.sqrt(0.299*a[0]**2 + 0.587*a[1]**2 + 0.114*a[2]**2)

[238 211 190]


In [7]:
# https://stackoverflow.com/questions/1903462/how-can-i-zip-sort-parallel-numpy-arrays
# I want to do something like this, generate an array (a) which can be easily sorted,
# then use the sort order to sort the original array (b) by the same method.
# Here, array (b) represents a single column of pixel data (r g b).
b = np.array([[1,2,3], [4,5,6], [7,8,9], [1,1,1], [2,2,2], [3,3,3]])
print(b.shape)
a = np.array([6,3,4,2,1,5])
print(a.argsort())
print(b[a.argsort()])

# Array (a) is calculated by manipulating and then summing the entries of array (b).
# This is difficult to do all at once, but we can manipulate each column of the Nx3 array
# and then sum the result along axis=1
b[:,0] = b[:,0] * 2
print(b)
print(b.sum(axis=1))

(6, 3)
[4 3 1 2 5 0]
[[2 2 2]
 [1 1 1]
 [4 5 6]
 [7 8 9]
 [3 3 3]
 [1 2 3]]
[[ 2  2  3]
 [ 8  5  6]
 [14  8  9]
 [ 2  1  1]
 [ 4  2  2]
 [ 6  3  3]]
[ 7 19 31  4  8 12]


In [8]:
from PIL import Image
#import random, sys
from time import time

def quicker_sort(pixels):
    pixels = np.array(pixels)
    pix_cpy = pixels.copy()
    pix_cpy = pix_cpy.sum(axis=1)
    return pixels[pix_cpy.argsort()]

def quick_sort(pixels):
    #Quicksort function that sorts pixels based on combined RGB values (R + B + G)
    if pixels == []:
        return pixels

    else:
        pivot = pixels[0]
        lesser = quick_sort([x for x in pixels[1:] if (x[0] + x[1] + x[2]) < (pivot[0] + pivot[1] + pivot[2])])
        greater = quick_sort([x for x in pixels[1:] if (x[0] + x[1] + x[2]) >= (pivot[0] + pivot[1] + pivot[2])])
        return lesser + [pivot] + greater

# def quick_sort_arr(pixels)
#     if pixels == np.array([]):
#         return pixels
#     else:
#         pivot = pixels[0]
#         lesser 

def sort_all(image):
    img = Image.open(image)
    img = img.convert('RGBA')
    new = Image.new('RGBA', img.size)
    pixels = np.asarray(img)
    #pixels = img.getdata()
    sorted_pixels = quicker_sort(pixels)
    #print(sorted_pixels)
    new.putdata(sorted_pixels)
    new.save('output-sortall-vectorized.png')
    return
 
def sort_all_pixels(image):
    #sorts every line of pixels
    print("Sorting all pixels.")
    
    print("Opening image...")
    tick = time()
    img = Image.open(image)
    img = img.convert('RGBA')
    print("Get data...")
    data = img.load()

    new = Image.new('RGBA', img.size)
    tock = time()
    print("Time to load image: ", tock-tick)
    
    tick = time()
    pixels = []
    sorted_pixels = []
    print("Getting pixels...")
    #Load all of the pixels into the pixels list
    for y in range(img.size[1]):
        pixels.append([])
        for x in range(img.size[0]):
            pixels[y].append(data[x, y])
    tock = time()
    print("Time to get pixels: ", tock-tick)

    tick = time()
    print("Vectorized sorting pixels...")
    for y in range(img.size[1]):
        sorted_pixels.append(quicker_sort(pixels[y]))
    #print(sorted_pixels)
    tock = time()
    print("Time to vectorized sort: ", tock-tick)
    
    tick = time()
    print("Placing pixels...")
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            #print(type(sorted_pixels[y][x]), "done")
            new.putpixel((x, y), tuple(sorted_pixels[y][x])) #apply the pixels to the new image
    #new.putdata(sorted_pixels)
    tock = time()
    print("Time to place pixels: ", tock-tick)
    print("Saving image...")
    new.save('output-sortall-vectorized.png')

tick = time()
#sys.setrecursionlimit(10000) #Increase the recursion depth limit. Without this, the script fails on larger images because quicksort recurses too much.
sort_all_pixels("/Users/aleph/Downloads/IMG_4634.JPEG")
tock = time()
print("{:.2f} seconds".format(tock-tick))
sort_all("/Users/aleph/Downloads/IMG_4634.JPEG")

Sorting all pixels.
Opening image...
Get data...
Time to load image:  0.0026979446411132812
Getting pixels...
Time to get pixels:  0.01676011085510254
Vectorized sorting pixels...
Time to vectorized sort:  0.022541046142578125
Placing pixels...
Time to place pixels:  0.15334606170654297
Saving image...
0.23 seconds


TypeError: color must be int or tuple

In [4]:
def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst
im1 = Image.open('./sonoshi_wp.png')
im2 = Image.open('./output-sortall-vectorized.png')
get_concat_v(im1, im2).save('./sonoshi_stack_storted.png')

In [ ]:
def sort_all_pixels(image):
    #sorts every line of pixels
    print("Sorting all pixels.")
    
    print("Opening image...")
    tick = time()
    img = Image.open(image)
    img = img.convert('RGBA')
    print("Get data...")
    data = img.load()

    new = Image.new('RGBA', img.size)
    tock = time()
    print("Time to load image: ", tock-tick)
    
    tick = time()
    pixels = []
    sorted_pixels = []
    print("Getting pixels...")
    #Load all of the pixels into the pixels list
    for y in range(img.size[1]):
        pixels.append([])
        for x in range(img.size[0]):
            pixels[y].append(data[x, y])
    tock = time()
    print("Time to get pixels: ", tock-tick)

    tick = time()
    print("Recursively sorting pixels...")
    for y in range(img.size[1]):
        sorted_pixels.append(quick_sort(pixels[y]))
    tock = time()
    print("Time to recursive sort: ", tock-tick)
    
    tick = time()
    print("Placing pixels...")
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            #print(type(sorted_pixels[y][x]), "done")
            new.putpixel((x, y), tuple(sorted_pixels[y][x])) #apply the pixels to the new image
    tock = time()
    print("Time to place pixels: ", tock-tick)
    print("Saving image...")
    new.save('output-sortall-recursive.png')
    
def random_sort_pixels(image, intensity):
    #sorts pixels in a random fashion

    if intensity > 100:
        intensity = 100

    print("Sorting " + image + " with intensity " + str(intensity))

    #Open the image, convert it to RGBA, get the pixels 
    print("Opening image...")
    img = Image.open(image)
    img = img.convert('RGBA')
    print("Get data...")
    data = img.load()

    new = Image.new('RGBA', img.size)

    pixels = []
    sorted_pixels = []
    print("Getting pixels...")
    #Load all of the pixels into the pixels list
    for y in range(img.size[1]):
        pixels.append([])
        for x in range(img.size[0]):
            pixels[y].append(data[x, y])

    print("Quicksorting pixels...")
    for y in range(img.size[1]):
        #pick different starting points for each line
        if(random.randint(0, 100) > intensity):
            sorted_pixels.append(pixels[y]) #Don't sort this line of pixels
        else:
            minsort = random.randint(3, len(pixels[y]) - 3) #pick the start of the sorted area on this pixel line
            maxsort = random.randint(minsort, len(pixels[y]) - 1)# pick the end of the sorted area on this pixel line
            sort = []
            for x in range(minsort, maxsort):
                sort.append(pixels[y][x]) 

            sort = quick_sort(sort) #sort the pixels by brightness

            i = 0
            for x in range(minsort, maxsort):
                pixels[y][x] = sort[i]
                i = i + 1

            sorted_pixels.append(pixels[y])


    print("Placing pixels...")
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            new.putpixel((x, y), sorted_pixels[y][x]) #apply the pixels to the new image

    print("Saving image...")
    new.save('output-sortrandom.png')

def sort_pixels_pivot(image):
    print ("Sorting pixels on pivot.")

    #Open the image, convert it to RGBA, get the pixels 
    print("Opening image...")
    img = Image.open(image)
    img = img.convert('RGBA')
    print("Get data...")
    data = img.load()

    new = Image.new('RGBA', img.size)

    pixels = []
    sorted_pixels = []
    print("Getting pixels...")
    #Load all of the pixels into the pixels list
    for y in range(img.size[1]):
        pixels.append([])
        for x in range(img.size[0]):
            pixels[y].append(data[x, y])

    print("Quicksorting pixels...")

    minsort = random.randint(3, img.size[0] - 3) #get sorting pivot
    for y in range(img.size[1]):
        maxsort = random.randint(minsort, len(pixels[y]) - 1) #pick the end of the sorted area on this pixel line
        sort = []
        for x in range(minsort, maxsort):
            sort.append(pixels[y][x])
        sort = quick_sort(sort) #sort the pixels by brightness

        i = 0
        for x in range(minsort, maxsort):
            pixels[y][x] = sort[i]
            i = i + 1

        sorted_pixels.append(pixels[y])


    print("Placing pixels...")
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            new.putpixel((x, y), sorted_pixels[y][x]) #apply the pixels to the new image

    print("Saving image...")
    new.save('output-sortpivot.png')
    
random_sort_pixels("/Users/sean/Downloads/spotted-lanternfly-nymphs-adults.jpg", 100)
sort_pixels_pivot("/Users/sean/Downloads/spotted-lanternfly-nymphs-adults.jpg")